In [2]:
import re
import csv
import json
from collections import defaultdict
from collections import OrderedDict
from collections import Counter
import numpy as np
import random
import pandas as pd
from pandas import Series
from pandas import DataFrame
from matplotlib import pyplot as plt

In [3]:
csvfile = pd.read_csv(open('EuCitiesTemperatures.csv'))

In [4]:
csvfile

,city,country,population,EU,coastline,latitude,longitude,temperature
0,Elbasan,Albania,2.90,no,yes,41.12,20.08,15.18
1,Andorra,Andorra,0.07,no,no,42.50,1.52,NaN
2,Innsbruck,Austria,8.57,yes,no,NaN,11.41,4.54
3,Graz,Austria,8.57,yes,no,47.08,NaN,6.91
4,Linz,Austria,8.57,yes,no,48.32,14.29,6.79
...,...,...,...,...,...,...,...,...
208,Edinburgh,United Kingdom,65.11,yes,yes,55.95,NaN,NaN
209,Exeter,United Kingdom,65.11,yes,yes,50.70,-3.53,10.14
210,Inverness,United Kingdom,65.11,yes,yes,NaN,-4.23,8.00
211,Glasgow,United Kingdom,65.11,yes,yes,55.87,-4.25,8.60


In [5]:
csvfile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213 entries, 0 to 212
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   city         213 non-null    object 
 1   country      213 non-null    object 
 2   population   213 non-null    float64
 3   EU           213 non-null    object 
 4   coastline    213 non-null    object 
 5   latitude     151 non-null    float64
 6   longitude    151 non-null    float64
 7   temperature  149 non-null    float64
dtypes: float64(4), object(4)
memory usage: 13.4+ KB


In [18]:
missingInfo = csvfile[csvfile['latitude'].isnull() | csvfile['longitude'].isnull()]
missingInfo

,city,country,population,EU,coastline,latitude,longitude,temperature
2,Innsbruck,Austria,8.57,yes,no,NaN,11.41,4.54
3,Graz,Austria,8.57,yes,no,47.08,NaN,6.91
5,Salzburg,Austria,8.57,yes,no,47.81,NaN,4.62
7,Minsk,Belarus,9.48,no,no,NaN,27.57,5.28
8,Orsha,Belarus,9.48,no,no,NaN,NaN,4.93
...,...,...,...,...,...,...,...,...
205,Birmingham,United Kingdom,65.11,yes,yes,NaN,-1.92,8.81
206,Bradford,United Kingdom,65.11,yes,yes,NaN,-1.75,NaN
207,Dundee,United Kingdom,65.11,yes,yes,56.47,NaN,NaN
208,Edinburgh,United Kingdom,65.11,yes,yes,55.95,NaN,NaN


In [22]:
set(csvfile['country'].values)

{'Albania',
 'Andorra',
 'Austria',
 'Belarus',
 'Belgium',
 'Bosnia and Herzegovina',
 'Bulgaria',
 'Croatia',
 'Czech Republic',
 'Denmark',
 'Estonia',
 'Finland',
 'France',
 'Germany',
 'Greece',
 'Hungary',
 'Ireland',
 'Italy',
 'Latvia',
 'Lithuania',
 'Macedonia',
 'Moldova',
 'Montenegro',
 'Netherlands',
 'Norway',
 'Poland',
 'Portugal',
 'Romania',
 'Serbia',
 'Slovakia',
 'Slovenia',
 'Spain',
 'Sweden',
 'Switzerland',
 'Turkey',
 'Ukraine',
 'United Kingdom'}

In [35]:
latsDict = {}
longsDict = {}
for country in set(csvfile['country'].values):
    subset = csvfile.loc[csvfile['country'] == country]
    latsDict[country] = round(subset['latitude'].mean(),2)
    longsDict[country] = round(subset['longitude'].mean(),2)
    
print(latsDict)
print(longsDict)
    
    
    
# avg = subset['latitude'].mean()
# subset


{'Ukraine': 48.71, 'Italy': 42.08, 'Belgium': 51.22, 'Germany': 50.59, 'Romania': 45.52, 'Moldova': 47.76, 'Serbia': 44.82, 'Bosnia and Herzegovina': 43.85, 'France': 47.13, 'Croatia': 45.33, 'Sweden': 60.88, 'Netherlands': 52.78, 'Bulgaria': 43.05, 'Finland': 61.78, 'Switzerland': 47.48, 'Hungary': 47.24, 'Turkey': 39.17, 'Slovenia': 46.06, 'United Kingdom': 53.55, 'Macedonia': 42.0, 'Spain': 39.95, 'Belarus': 52.61, 'Greece': 38.2, 'Montenegro': 42.47, 'Slovakia': 48.44, 'Denmark': 55.4, 'Portugal': 38.72, 'Andorra': 42.5, 'Albania': 41.12, 'Austria': 47.85, 'Poland': 52.48, 'Latvia': 56.95, 'Estonia': 59.43, 'Czech Republic': 49.7, 'Lithuania': 55.72, 'Ireland': 53.3, 'Norway': 62.39}
{'Ukraine': 32.24, 'Italy': 13.42, 'Belgium': 3.23, 'Germany': 10.05, 'Romania': 25.96, 'Moldova': 28.86, 'Serbia': 20.88, 'Bosnia and Herzegovina': 13.38, 'France': 3.43, 'Croatia': 15.46, 'Sweden': 17.36, 'Netherlands': 5.73, 'Bulgaria': 26.35, 'Finland': 23.64, 'Switzerland': 7.43, 'Hungary': 19.8, 

In [62]:
missingInfo

,city,country,population,EU,coastline,latitude,longitude,temperature
2,Innsbruck,Austria,8.57,yes,no,NaN,11.41,4.54
3,Graz,Austria,8.57,yes,no,47.08,NaN,6.91
5,Salzburg,Austria,8.57,yes,no,47.81,NaN,4.62
7,Minsk,Belarus,9.48,no,no,NaN,27.57,5.28
8,Orsha,Belarus,9.48,no,no,NaN,NaN,4.93
...,...,...,...,...,...,...,...,...
205,Birmingham,United Kingdom,65.11,yes,yes,NaN,-1.92,8.81
206,Bradford,United Kingdom,65.11,yes,yes,NaN,-1.75,NaN
207,Dundee,United Kingdom,65.11,yes,yes,56.47,NaN,NaN
208,Edinburgh,United Kingdom,65.11,yes,yes,55.95,NaN,NaN


In [67]:
for country, avg in latsDict.items():
    
    missingInfo.loc[tmp['country'] == country,'latitude'] =  missingInfo.loc[tmp['country'] == country,'latitude'].fillna(latsDict[country])
    missingInfo.loc[tmp['country'] == country,'longitude'] =  missingInfo.loc[tmp['country'] == country,'longitude'].fillna(longsDict[country])
missingInfo

,city,country,population,EU,coastline,latitude,longitude,temperature
2,Innsbruck,Austria,8.57,yes,no,47.85,11.41,4.54
3,Graz,Austria,8.57,yes,no,47.08,14.02,6.91
5,Salzburg,Austria,8.57,yes,no,47.81,14.02,4.62
7,Minsk,Belarus,9.48,no,no,52.61,27.57,5.28
8,Orsha,Belarus,9.48,no,no,52.61,25.64,4.93
...,...,...,...,...,...,...,...,...
205,Birmingham,United Kingdom,65.11,yes,yes,53.55,-1.92,8.81
206,Bradford,United Kingdom,65.11,yes,yes,53.55,-1.75,NaN
207,Dundee,United Kingdom,65.11,yes,yes,56.47,-2.94,NaN
208,Edinburgh,United Kingdom,65.11,yes,yes,55.95,-2.94,NaN


In [71]:
for country, avg in latsDict.items():
    
    csvfile.loc[csvfile['country'] == country,'latitude'] =  csvfile.loc[csvfile['country'] == country,'latitude'].fillna(latsDict[country])
    csvfile.loc[csvfile['country'] == country,'longitude'] =  csvfile.loc[csvfile['country'] == country,'longitude'].fillna(longsDict[country])
csvfile

,city,country,population,EU,coastline,latitude,longitude,temperature
0,Elbasan,Albania,2.90,no,yes,41.12,20.08,15.18
1,Andorra,Andorra,0.07,no,no,42.50,1.52,NaN
2,Innsbruck,Austria,8.57,yes,no,47.85,11.41,4.54
3,Graz,Austria,8.57,yes,no,47.08,14.02,6.91
4,Linz,Austria,8.57,yes,no,48.32,14.29,6.79
...,...,...,...,...,...,...,...,...
208,Edinburgh,United Kingdom,65.11,yes,yes,55.95,-2.94,NaN
209,Exeter,United Kingdom,65.11,yes,yes,50.70,-3.53,10.14
210,Inverness,United Kingdom,65.11,yes,yes,53.55,-4.23,8.00
211,Glasgow,United Kingdom,65.11,yes,yes,55.87,-4.25,8.60


In [73]:
csvfile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213 entries, 0 to 212
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   city         213 non-null    object 
 1   country      213 non-null    object 
 2   population   213 non-null    float64
 3   EU           213 non-null    object 
 4   coastline    213 non-null    object 
 5   latitude     213 non-null    float64
 6   longitude    213 non-null    float64
 7   temperature  149 non-null    float64
dtypes: float64(4), object(4)
memory usage: 13.4+ KB
